In [32]:
import pandas as pd
import numpy as np
import os

In [33]:
bakrep_metadata = pd.read_csv("/Users/ktruong/workspace/Workspace/jupyter_notebook/data/bakrep-export_18112024.tsv",sep='\t', low_memory=False, index_col="#id")

In [34]:
[tup for tup in zip(bakrep_metadata.columns, bakrep_metadata.dtypes)]

[('bakta.genome.genus', dtype('O')),
 ('bakta.genome.species', dtype('O')),
 ('bakta.genome.strain', dtype('O')),
 ('bakta.stats.no_sequences', dtype('float64')),
 ('bakta.stats.size', dtype('float64')),
 ('bakta.stats.gc', dtype('float64')),
 ('bakta.stats.n_ratio', dtype('float64')),
 ('bakta.stats.n50', dtype('float64')),
 ('bakta.stats.coding_ratio', dtype('float64')),
 ('gtdbtk.classification.domain', dtype('O')),
 ('gtdbtk.classification.phylum', dtype('O')),
 ('gtdbtk.classification.class', dtype('O')),
 ('gtdbtk.classification.order', dtype('O')),
 ('gtdbtk.classification.family', dtype('O')),
 ('gtdbtk.classification.genus', dtype('O')),
 ('gtdbtk.classification.species', dtype('O')),
 ('mlst.sequence_type', dtype('O')),
 ('checkm2.quality.completeness', dtype('float64')),
 ('checkm2.quality.contamination', dtype('float64')),
 ('metadata.accession', dtype('O')),
 ('metadata.collected_by', dtype('O')),
 ('metadata.collection_date', dtype('O')),
 ('metadata.country', dtype('O'))

In [48]:
species_groups =  bakrep_metadata.groupby(['gtdbtk.classification.species']).groups
group_sizes = bakrep_metadata.groupby(['gtdbtk.classification.species']).size()
group_sizes_with_na = bakrep_metadata.groupby(['gtdbtk.classification.species'], dropna=False).size()

In [36]:
highly_sampled_species = group_sizes[group_sizes>=100]
highly_sampled_species.sum()

632264

In [37]:
dustbins = group_sizes[group_sizes<100]

In [62]:
dustbins.sum()

25485

In [49]:
group_sizes_with_na.sum()

661402

In [52]:
bakrep_metadata['gtdbtk.classification.species'].isna().sum()

3653

In [53]:
632264 + 25485 + 3653

661402

In [72]:
genomes_without_species = bakrep_metadata[bakrep_metadata['gtdbtk.classification.species'].isna()]
genomes_without_species.index.values

array(['SAMD00000585', 'SAMD00000612', 'SAMD00000644', ...,
       'SAMN10342921', 'SAMN10389991', 'SAMN10390024'], dtype=object)

In [73]:
dustbin_genomes = list(genomes_without_species.index.values)

In [74]:
for spc in dustbins.index.values:
    dustbin_genomes = dustbin_genomes + list(species_groups[spc].values)

In [76]:
len(dustbin_genomes)

29138

In [80]:
with open('species_clusters/dustbins.txt', "w") as f:
        for gen in dustbin_genomes:
            f.write(f"{gen}\n")

In [83]:
highly_sampled_species[group_sizes<=10000].sort_values()

gtdbtk.classification.species
Lactococcus lactis             100
Haemophilus influenzae_D       100
Burkholderia cenocepacia_B     100
Mesorhizobium sp003996995      100
Enterobacter kobei             104
                              ... 
Vibrio cholerae               5562
Pseudomonas aeruginosa        6286
Enterococcus_B faecium        8284
Campylobacter_D coli          8849
Neisseria gonorrhoeae         8904
Length: 127, dtype: int64

In [84]:
fewer_than_10k = highly_sampled_species[group_sizes<=10000]
for spc in fewer_than_10k.index.values:
    genomes = list(species_groups[spc].values)
    with open(f"species_clusters/fewer_than_10k/{spc.replace(" ","_")}.txt","w") as f:
        for gen in genomes:
            f.write(f"{gen}\n")

In [85]:
more_than_10k = highly_sampled_species[group_sizes>=10000]
for spc in more_than_10k.index.values:
    genomes = list(species_groups[spc].values)
    with open(f"species_clusters/more_than_10k/{spc.replace(" ","_")}.txt","w") as f:
        for gen in genomes:
            f.write(f"{gen}\n")

In [87]:
fewer_than_10k.sum() + more_than_10k.sum() + len(dustbin_genomes)

661402

In [89]:
more_than_10k.sort_values()

gtdbtk.classification.species
Streptococcus agalactiae       10359
Listeria monocytogenes_B       12244
Listeria monocytogenes         12245
Clostridioides difficile       13579
Klebsiella pneumoniae          13637
Streptococcus pyogenes         16821
Neisseria meningitidis         17153
Campylobacter_D jejuni         28336
Staphylococcus aureus          47997
Mycobacterium tuberculosis     49020
Streptococcus pneumoniae       51433
Escherichia coli               89136
Salmonella enterica           178585
dtype: int64